In [32]:
import pandas as pd
from collections import Counter
import numpy as np
from copy import deepcopy
import random
from functools import reduce
import pickle

In [33]:
train_replace_dict = {'atis_flight#atis_airfare': 'atis_airfare', 
                                     'atis_ground_service#atis_ground_fare': 'atis_ground_fare',
                                     'atis_aircraft#atis_flight#atis_flight_no': 'atis_flight_no',
                                     'atis_airfare#atis_flight_time':'atis_flight_time',
                                     'atis_airline#atis_flight_no':'atis_flight_no',
                                     'atis_flight_no#atis_airline' : 'atis_flight_no',
                                     'atis_airfare#atis_flight': 'atis_airfare',
                                     'atis_flight#atis_airline' : 'atis_airline'}

# intent to idx mapping
intent_list = (pd.read_csv('../data/multiATIS/intent_list.csv',sep=',',header=None,names=['INTENT']).INTENT.values.tolist())
intent2idx = { intent:idx for idx,intent in enumerate(intent_list)}

# slot idx mapping
final_slots = pd.read_csv('../data/multiATIS/slots_list.csv',sep=',',header=None,names=['SLOTS']).SLOTS.values.tolist()
slots2idx  = {slots:idx for idx,slots in enumerate(final_slots)}
idx2slots = {v: k for k, v in slots2idx.items()}

In [34]:
def mapping2idx(df,fn):
    
    #del df['id']

    # adding intent encoding
    encoded_intent = []
    for intent in list(df.INTENT):
        try:
            encoded_intent.append(intent2idx[intent])
        except:
            print(intent)
            encoded_intent.append(intent2idx['OOI'])
            
        
    df['INTENT_ID'] = encoded_intent 

    # adding mapped slot column
    slots_ID = []

    for annotations in list(df.SLOTS):
        encoded_slot = ""
        #print((annotations.split()))
        for tokens in annotations.split():
            try: 
                encoded_slot += str(slots2idx[tokens]) + " "
            except:
                print('token',tokens,fn)
                encoded_slot += str(slots2idx['O'])
        slots_ID.append(" ".join(encoded_slot.split()))

    df['SLOTS_ID'] = slots_ID
    
    return df

In [35]:
def spoken_aug(data,tau,aug_count):
    
    orig_data = deepcopy(data)
    
    augINTENT, augSLOTS, augTEXT,augID = [],[],[],[]
    cnt = 0
    for sample in data.values.tolist():
        
        sent_length = len(sample[1].split(' '))
        
        if sent_length <=5:
            continue
        for t in tau:
            for _ in range(aug_count):
            
                del_count = 1 if t*sent_length <=1.0 else int(t*sent_length)
                del_index = random.sample(list(range(sent_length)),del_count)
                
                augINTENT.append(sample[3])
                TEXT = ' '.join([i for j, i in enumerate(sample[1].split(' ')) if j not in del_index])
                SLOTS = ' '.join([i for j, i in enumerate(sample[2].split(' ')) if j not in del_index])
                augTEXT.append(TEXT)
                augSLOTS.append(SLOTS)
                augID.append(cnt)
                cnt+=1
    
    #print(len(augTEXT),augTEXT[:5])            
    augPD = pd.DataFrame([augID,augTEXT,augSLOTS,augINTENT],index=['ID','TEXT','SLOTS','INTENT']).T
    
    return augPD

In [44]:
def process_data(data_path,out_path,fn,aug=True,merge=True):
    
    # loading dataset
    data = pd.read_csv(data_path,sep='\t',header=0,names=['ID','TEXT','SLOTS','INTENT'])
    
    # lowerCasing the TEXT column
    data['TEXT'] = data['TEXT'].str.lower()
    
    # handling multi-label instances 
    data['INTENT'] = data['INTENT'].replace(train_replace_dict)
    
    if aug:
        aug_data = spoken_aug(data,tau=[0.25,0.50,0.75],aug_count=5)
    
    #print(aug_data.INTENT)
    
    data = mapping2idx(data,fn)
    aug_data = mapping2idx(aug_data,fn)
    
    #print(len(data),len(aug_data))
    
    if merge:
        data_final = pd.concat([data,aug_data])
    
    data_final = data_final.sample(frac=1).reset_index(drop=True)
    
    
    
    data_final.to_csv(out_path,sep='\t',index=False)
    

In [45]:
# processing EN training data
process_data('../data/multiATIS/raw/train_dev_test/train_EN.tsv','../data/multiATIS/splits/baseline/train_EN.tsv','EN')
# processing ES training data
process_data('../data/multiATIS/raw/train_dev_test/train_ES.tsv','../data/multiATIS/splits/baseline/train_ES.tsv','ES')
# processing DE training data
process_data('../data/multiATIS/raw/train_dev_test/train_DE.tsv','../data/multiATIS/splits/baseline/train_DE.tsv','DE')
# processing FR training data
process_data('../data/multiATIS/raw/train_dev_test/train_FR.tsv','../data/multiATIS/splits/baseline/train_FR.tsv','FR')

In [46]:
# processing EN training data
process_data('../data/multiATIS/raw/train_dev_test/dev_EN.tsv','../data/multiATIS/splits/baseline/dev_EN.tsv','EN')
# processing ES training data
process_data('../data/multiATIS/raw/train_dev_test/dev_ES.tsv','../data/multiATIS/splits/baseline/dev_ES.tsv','ES')
# processing DE training data
process_data('../data/multiATIS/raw/train_dev_test/dev_DE.tsv','../data/multiATIS/splits/baseline/dev_DE.tsv','DE')
# processing FR training data
process_data('../data/multiATIS/raw/train_dev_test/dev_FR.tsv','../data/multiATIS/splits/baseline/dev_FR.tsv','FR')

In [47]:
# processing EN training data
process_data('../data/multiATIS/raw/train_dev_test/test_EN.tsv','../data/multiATIS/splits/baseline/test_EN.tsv','EN')
# processing ES training data
process_data('../data/multiATIS/raw/train_dev_test/test_ES.tsv','../data/multiATIS/splits/baseline/test_ES.tsv','ES')
# processing DE training data
process_data('../data/multiATIS/raw/train_dev_test/test_DE.tsv','../data/multiATIS/splits/baseline/test_DE.tsv','DE')
# processing FR training data
process_data('../data/multiATIS/raw/train_dev_test/test_FR.tsv','../data/multiATIS/splits/baseline/test_FR.tsv','FR')

atis_day_name
atis_day_name
token B-compartment EN
token B-stoploc.airport_code EN
token B-booking_class EN
token B-flight EN
atis_day_name
atis_day_name
atis_day_name
atis_day_name
atis_day_name
atis_day_name
atis_day_name
atis_day_name
atis_day_name
atis_day_name
atis_day_name
atis_day_name
atis_day_name
atis_day_name
atis_day_name
atis_day_name
atis_day_name
atis_day_name
atis_day_name
atis_day_name
atis_day_name
atis_day_name
atis_day_name
atis_day_name
atis_day_name
atis_day_name
atis_day_name
atis_day_name
atis_day_name
atis_day_name
token B-compartment EN
token B-compartment EN
token B-compartment EN
token B-compartment EN
token B-compartment EN
token B-compartment EN
token B-compartment EN
token B-compartment EN
token B-compartment EN
token B-compartment EN
token B-compartment EN
token B-stoploc.airport_code EN
token B-stoploc.airport_code EN
token B-stoploc.airport_code EN
token B-stoploc.airport_code EN
token B-stoploc.airport_code EN
token B-stoploc.airport_code EN
token B-s